# Scrapping LinkedIn profiles

In [11]:
import os
import requests
import re
import pandas as pd

# Fuction that scrapes LinkedIn profiles

In [19]:
def scrape_linkedin_profile(linkedin_profile_url: str):
    """scrape information from LinkedIn profiles,
    Manually scrape the information from the LinkedIn profile"""
    api_key = "2ZjHkun0Sq6KGa_o-4p7jg"
    header_dic = {"Authorization": f"Bearer {api_key}"}
    api_endpoint = "https://nubela.co/proxycurl/api/v2/linkedin"

    response = requests.get(
        api_endpoint, params={"url": linkedin_profile_url}, headers=header_dic
    )

    data = response.json()
    data = {
        k: v
        for k, v in data.items()
        if v not in ([], "", "", None)
        and k
        not in [
            "people_also_viewed",
            "recommendations",
            "similarly_named_profiles",
            "articles",
            "background_cover_image_url",
            "activities",
            "volunteer_work",
        ]
    }
    if data.get("groups"):
        for group_dict in data.get("groups"):
            group_dict.pop("profile_pic_url")

    return data

# This function manually scrapes a LinkedIn profile, taking the person's URL as argument

# Scraping and cleaning our dataframes

## First dataframe: 2015-2016 class

In [ ]:
scrap = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2015-2016.csv') 

scrap = scrap[scrap['Link perfil'] != 'Link no existente']

scrap = scrap.reset_index()

scrap = scrap.drop('index', axis=1)

# We open our csv, containing the full name and URL of the profile of our students

In [43]:
scrap["Datos escrapeados"] = scrap["Link perfil"].apply(scrape_linkedin_profile)

# We apply our function to our CSV


In [66]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = scrap.loc[scrap['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

scrap = scrap.drop(indices_a_eliminar)

# And we take them out 


In [337]:
copi = scrap.copy() # We create a copy of our original, dataframe, to use it as test

In [338]:
copi = copi.drop(['index', 'level_0'], axis=1)

copi


,Grado,Nombre,Primer apellido,Segundo apellido,Email universitario,Email personal,Nombre completo,Link perfil,Datos escrapeados
0,Comunicación audiovisual y Publicidad,Antonio,Cortés,Ruiz,a.cortes7@usp.ceu.es,tonius_92@hotmail.com,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,{'public_identifier': 'antonio-cort%C3%A9s-rui...
1,Comunicación audiovisual,José Antonio,Quero,González,ja.quero@usp.ceu.es,yosean_7@hotmail.com,José Antonio Quero González,https://www.linkedin.com/in/jos%C3%A9-antonio-...,{'public_identifier': 'josé-antonio-quero-gonz...
2,Comunicación audiovisual y Periodismo,Noelia,Fernández,Aceituno,n.fernandez24@usp.ceu.es,noelia_fernandezaceituno@hotmail.com,Noelia Fernández Aceituno,https://www.linkedin.com/in/noelia-fern%C3%A1n...,{'public_identifier': 'noelia-fern%C3%A1ndez-a...
3,Comunicación audiovisual,Laura,Rubio,Valladolid,l.rubio13@usp.ceu.es,laurarv_12@hotmail.com,Laura Rubio Valladolid,https://www.linkedin.com/in/laura-rubio-vallad...,{'public_identifier': 'laura-rubio-valladolid-...
4,Comunicación audiovisual y Periodismo,Pablo,Seara,García,p.seara@usp.ceu.es,cgsobrado@telefonica.net,Pablo Seara García,https://www.linkedin.com/in/pablo-garcia-de-le...,"{'public_identifier': 'pablo-garcia-de-leon', ..."
...,...,...,...,...,...,...,...,...,...
233,Comunicación digital,Lucía,Merino,Pérez,l.merino6@usp.ceu.es,luciamp94@gmail.com,Lucía Merino Pérez,https://www.linkedin.com/in/luciamerinoperez?m...,"{'public_identifier': 'luciamerinoperez', 'pro..."
234,Comunicación digital,Carolina,Fernández,Moreno,c.fernandez102@usp.ceu.es,carolina_fernandez_1@hotmail.com,Carolina Fernández Moreno,https://www.linkedin.com/in/carolina-fern%C3%A...,{'public_identifier': 'carolina-fernández-more...
235,Comunicación digital,Jorge,Mariscal,Ávila,j.mariscal@usp.ceu.es,jorgemariscal94@gmail.com,Jorge Mariscal Ávila,https://www.linkedin.com/in/jorgemariscalavila...,"{'public_identifier': 'jorgemariscalavila', 'f..."
236,Comunicación digital,Mariano,Gómez,Labrador,m.gomez77@usp.ceu.es,mgl1991@hotmail.com,Mariano Gómez Labrador,https://www.linkedin.com/in/mariano-g%C3%B3mez...,{'public_identifier': 'mariano-g%C3%B3mez-labr...


In [339]:
copi2 = copi.copy()

copi2 =  copi2.reset_index()

copi2 = copi2.drop('index', axis=1)

copi2

,Grado,Nombre,Primer apellido,Segundo apellido,Email universitario,Email personal,Nombre completo,Link perfil,Datos escrapeados
0,Comunicación audiovisual y Publicidad,Antonio,Cortés,Ruiz,a.cortes7@usp.ceu.es,tonius_92@hotmail.com,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,{'public_identifier': 'antonio-cort%C3%A9s-rui...
1,Comunicación audiovisual,José Antonio,Quero,González,ja.quero@usp.ceu.es,yosean_7@hotmail.com,José Antonio Quero González,https://www.linkedin.com/in/jos%C3%A9-antonio-...,{'public_identifier': 'josé-antonio-quero-gonz...
2,Comunicación audiovisual y Periodismo,Noelia,Fernández,Aceituno,n.fernandez24@usp.ceu.es,noelia_fernandezaceituno@hotmail.com,Noelia Fernández Aceituno,https://www.linkedin.com/in/noelia-fern%C3%A1n...,{'public_identifier': 'noelia-fern%C3%A1ndez-a...
3,Comunicación audiovisual,Laura,Rubio,Valladolid,l.rubio13@usp.ceu.es,laurarv_12@hotmail.com,Laura Rubio Valladolid,https://www.linkedin.com/in/laura-rubio-vallad...,{'public_identifier': 'laura-rubio-valladolid-...
4,Comunicación audiovisual y Periodismo,Pablo,Seara,García,p.seara@usp.ceu.es,cgsobrado@telefonica.net,Pablo Seara García,https://www.linkedin.com/in/pablo-garcia-de-le...,"{'public_identifier': 'pablo-garcia-de-leon', ..."
...,...,...,...,...,...,...,...,...,...
233,Comunicación digital,Lucía,Merino,Pérez,l.merino6@usp.ceu.es,luciamp94@gmail.com,Lucía Merino Pérez,https://www.linkedin.com/in/luciamerinoperez?m...,"{'public_identifier': 'luciamerinoperez', 'pro..."
234,Comunicación digital,Carolina,Fernández,Moreno,c.fernandez102@usp.ceu.es,carolina_fernandez_1@hotmail.com,Carolina Fernández Moreno,https://www.linkedin.com/in/carolina-fern%C3%A...,{'public_identifier': 'carolina-fernández-more...
235,Comunicación digital,Jorge,Mariscal,Ávila,j.mariscal@usp.ceu.es,jorgemariscal94@gmail.com,Jorge Mariscal Ávila,https://www.linkedin.com/in/jorgemariscalavila...,"{'public_identifier': 'jorgemariscalavila', 'f..."
236,Comunicación digital,Mariano,Gómez,Labrador,m.gomez77@usp.ceu.es,mgl1991@hotmail.com,Mariano Gómez Labrador,https://www.linkedin.com/in/mariano-g%C3%B3mez...,{'public_identifier': 'mariano-g%C3%B3mez-labr...


In [340]:
from pandas import json_normalize

normalized_data = pd.json_normalize(copi2['Datos escrapeados']) 

# We create one column per key in our 'Datos escrapeados' column


In [341]:
copi2 = pd.concat([copi2, normalized_data], axis=1) # We apply those columns to our copi2 dataframe


In [342]:
columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'city', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']  

In [343]:
copi2 = copi2.drop(columns=columnas_a_eliminar)

# We get rid of our unwanted columns

In [344]:
copi2 = copi2.explode('experiences')

# We explode our lists in 'experiences' column, to have as many rows per student as experiences she/he has

In [345]:
copi2 = copi2.reset_index()

copi2 = copi2.drop('index', axis=1)

copi2 # We reset our indexes, to make it more clear

,Grado,Nombre,Primer apellido,Segundo apellido,Nombre completo,Link perfil,experiences
0,Comunicación audiovisual y Publicidad,Antonio,Cortés,Ruiz,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,"{'starts_at': {'day': 1, 'month': 10, 'year': ..."
1,Comunicación audiovisual y Publicidad,Antonio,Cortés,Ruiz,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,"{'starts_at': {'day': 1, 'month': 2, 'year': 2..."
2,Comunicación audiovisual y Publicidad,Antonio,Cortés,Ruiz,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,"{'starts_at': {'day': 1, 'month': 6, 'year': 2..."
3,Comunicación audiovisual y Publicidad,Antonio,Cortés,Ruiz,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,"{'starts_at': {'day': 1, 'month': 9, 'year': 2..."
4,Comunicación audiovisual y Publicidad,Antonio,Cortés,Ruiz,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,"{'starts_at': {'day': 1, 'month': 4, 'year': 2..."
...,...,...,...,...,...,...,...
1549,Comunicación digital,Alvaro,Diaz,Fernandez,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,"{'starts_at': {'day': 1, 'month': 12, 'year': ..."
1550,Comunicación digital,Alvaro,Diaz,Fernandez,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,"{'starts_at': {'day': 1, 'month': 12, 'year': ..."
1551,Comunicación digital,Alvaro,Diaz,Fernandez,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,"{'starts_at': {'day': 1, 'month': 3, 'year': 2..."
1552,Comunicación digital,Alvaro,Diaz,Fernandez,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,"{'starts_at': {'day': 1, 'month': 9, 'year': 2..."


In [346]:
normalized_data1 = pd.json_normalize(copi2['experiences']) 


copi2 = pd.concat([copi2, normalized_data1], axis=1) # We apply those columns to our copi2 dataframe


In [347]:
cols_a_eliminar = ['Primer apellido', 'Nombre', 'Segundo apellido', 'location', 'experiences', 'company_linkedin_profile_url', 'description', 'logo_url', 'ends_at', 'starts_at']                  
                   
copi2 = copi2.drop(cols_a_eliminar, axis=1)

copi2   


# We get rid of unwanted columns

,Grado,Nombre,Nombre completo,Link perfil,company,title,starts_at.day,starts_at.month,starts_at.year,ends_at.day,ends_at.month,ends_at.year
0,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,TBWA\ España,Junior Art Director/ Crearive,1.0,10.0,2018.0,31.0,7.0,2019.0
1,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,ContrapuntoBBDO,Junior Art Director / Creative,1.0,2.0,2018.0,30.0,9.0,2018.0
2,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Publicis Groupe,Junior Art Director / Creative,1.0,6.0,2016.0,31.0,12.0,2016.0
3,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Längstrump,Junior Art Director / Creative,1.0,9.0,2015.0,29.0,2.0,2016.0
4,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Grupo Boomerang TV,Ayudante de Producción,1.0,4.0,2015.0,31.0,7.0,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1549,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,easylife conciliación,Gestor de proyectos - Carta de Beneficios,1.0,12.0,2020.0,30.0,9.0,2021.0
1550,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,easylife conciliación,Especialista en comunicación corporativa y org...,1.0,12.0,2016.0,30.0,6.0,2021.0
1551,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,"APP Android: Investiga Coches, Subastas Agenci...",Especialista en posicionamiento web,1.0,3.0,2016.0,31.0,12.0,2016.0
1552,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,Grupo Trivium IC,Redactor de contenidos web,1.0,9.0,2015.0,31.0,1.0,2016.0


In [351]:
copi2.isnull().sum()

Grado              0
Nombre             0
Nombre completo    0
Link perfil        0
company            0
title              0
starts_at.day      0
starts_at.month    0
starts_at.year     0
ends_at.day        0
ends_at.month      0
ends_at.year       0
dtype: int64

In [349]:
copi2['ends_at.day'].fillna('actualidad', inplace=True)
copi2['ends_at.month'].fillna('actualidad', inplace=True)
copi2['ends_at.year'].fillna('actualidad', inplace=True)

# If any of our ending columns have nulls, it means that that person is still working there

In [350]:
copi2.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

# We get rid of all our experiences whose start date is null

In [332]:
copi2['starts_at.day'] = pd.to_datetime(copi2['starts_at.day'])
copi2['starts_at.month'] = pd.to_datetime(copi2['starts_at.month'])
copi2['starts_at.year'] = pd.to_datetime(copi2['starts_at.year'])

copi2['ends_at.day'] = pd.to_datetime(copi2['ends_at.day'], errors='coerce')
copi2['ends_at.month'] = pd.to_datetime(copi2['ends_at.month'], errors='coerce')
copi2['ends_at.year'] = pd.to_datetime(copi2['ends_at.year'], errors='coerce')

# We convert all dates to datetime format


In [352]:
copi2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1495 entries, 0 to 1553
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Grado            1495 non-null   object 
 1   Nombre           1495 non-null   object 
 2   Nombre completo  1495 non-null   object 
 3   Link perfil      1495 non-null   object 
 4   company          1495 non-null   object 
 5   title            1495 non-null   object 
 6   starts_at.day    1495 non-null   float64
 7   starts_at.month  1495 non-null   float64
 8   starts_at.year   1495 non-null   float64
 9   ends_at.day      1495 non-null   object 
 10  ends_at.month    1495 non-null   object 
 11  ends_at.year     1495 non-null   object 
dtypes: float64(3), object(9)
memory usage: 151.8+ KB


In [353]:
copi2

,Grado,Nombre,Nombre completo,Link perfil,company,title,starts_at.day,starts_at.month,starts_at.year,ends_at.day,ends_at.month,ends_at.year
0,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,TBWA\ España,Junior Art Director/ Crearive,1.0,10.0,2018.0,31.0,7.0,2019.0
1,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,ContrapuntoBBDO,Junior Art Director / Creative,1.0,2.0,2018.0,30.0,9.0,2018.0
2,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Publicis Groupe,Junior Art Director / Creative,1.0,6.0,2016.0,31.0,12.0,2016.0
3,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Längstrump,Junior Art Director / Creative,1.0,9.0,2015.0,29.0,2.0,2016.0
4,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Grupo Boomerang TV,Ayudante de Producción,1.0,4.0,2015.0,31.0,7.0,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1549,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,easylife conciliación,Gestor de proyectos - Carta de Beneficios,1.0,12.0,2020.0,30.0,9.0,2021.0
1550,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,easylife conciliación,Especialista en comunicación corporativa y org...,1.0,12.0,2016.0,30.0,6.0,2021.0
1551,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,"APP Android: Investiga Coches, Subastas Agenci...",Especialista en posicionamiento web,1.0,3.0,2016.0,31.0,12.0,2016.0
1552,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,Grupo Trivium IC,Redactor de contenidos web,1.0,9.0,2015.0,31.0,1.0,2016.0


In [377]:
copi3 = copi2.copy() # CREAMOS OTRO DATAFRAME DE COPIA PARA EXPERIMENTAR CON LAS FECHAS

In [378]:
import pandas as pd
import datetime

# 1. Combina las columnas de inicio en 'Fecha inicio' y fin en 'Fecha fin'
copi3['Fecha inicio'] = copi3.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi3['Fecha fin'] = copi3.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

# 2. Calcula la duración en meses
copi3['Duración (meses)'] = copi3.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

# 3. Reemplaza los valores de 'Fecha fin' con 'actualidad' cuando corresponda
copi3['Fecha fin'] = copi3.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

# Elimina las columnas intermedias (si es necesario)
copi3.drop(['starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)


In [386]:
copi3.rename(columns={'company': 'Empresa', 'title': 'Puesto'}, inplace=True)


In [391]:
copi3 = copi3.reset_index()

copi3 = copi3.drop('index', axis=1)

copi3


,Grado,Nombre,Nombre completo,Link perfil,Empresa,Puesto,Fecha inicio,Fecha fin,Duración (meses)
0,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,TBWA\ España,Junior Art Director/ Crearive,2018-10-01,2019-07-01 00:00:00,9
1,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,ContrapuntoBBDO,Junior Art Director / Creative,2018-02-01,2018-09-01 00:00:00,7
2,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Publicis Groupe,Junior Art Director / Creative,2016-06-01,2016-12-01 00:00:00,6
3,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Längstrump,Junior Art Director / Creative,2015-09-01,2016-02-01 00:00:00,5
4,Comunicación audiovisual y Publicidad,Antonio,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Grupo Boomerang TV,Ayudante de Producción,2015-04-01,2015-07-01 00:00:00,3
...,...,...,...,...,...,...,...,...,...
1490,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,easylife conciliación,Gestor de proyectos - Carta de Beneficios,2020-12-01,2021-09-01 00:00:00,9
1491,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,easylife conciliación,Especialista en comunicación corporativa y org...,2016-12-01,2021-06-01 00:00:00,54
1492,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,"APP Android: Investiga Coches, Subastas Agenci...",Especialista en posicionamiento web,2016-03-01,2016-12-01 00:00:00,9
1493,Comunicación digital,Alvaro,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,Grupo Trivium IC,Redactor de contenidos web,2015-09-01,2016-01-01 00:00:00,4


In [ ]:
copi3 = copi3.drop('Nombre', axis=1)


In [399]:
copi3.to_csv('/Users/davidfernandez/Desktop/clean/experience/2015-2016.csv', index=False) 


In [400]:
copi3

,Grado,Nombre completo,Link perfil,Empresa,Puesto,Fecha inicio,Fecha fin,Duración (meses)
0,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,TBWA\ España,Junior Art Director/ Crearive,2018-10-01,2019-07-01 00:00:00,9
1,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,ContrapuntoBBDO,Junior Art Director / Creative,2018-02-01,2018-09-01 00:00:00,7
2,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Publicis Groupe,Junior Art Director / Creative,2016-06-01,2016-12-01 00:00:00,6
3,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Längstrump,Junior Art Director / Creative,2015-09-01,2016-02-01 00:00:00,5
4,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Grupo Boomerang TV,Ayudante de Producción,2015-04-01,2015-07-01 00:00:00,3
...,...,...,...,...,...,...,...,...
1490,Comunicación digital,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,easylife conciliación,Gestor de proyectos - Carta de Beneficios,2020-12-01,2021-09-01 00:00:00,9
1491,Comunicación digital,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,easylife conciliación,Especialista en comunicación corporativa y org...,2016-12-01,2021-06-01 00:00:00,54
1492,Comunicación digital,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,"APP Android: Investiga Coches, Subastas Agenci...",Especialista en posicionamiento web,2016-03-01,2016-12-01 00:00:00,9
1493,Comunicación digital,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,Grupo Trivium IC,Redactor de contenidos web,2015-09-01,2016-01-01 00:00:00,4
